# **Predictive Maintenance Datamart Notebook**

**📝 Dataset Overview:**  

These datasets supports the analysis of machine performance and predictiive maintenance by integrating sensor telemetry, error logs, maintenance actions, and failure events. The analysis can help predict failures, optimize maintenance schedules and reduce machine downtime.

**📝 Data Source:**  

The source datasets include:

| Source Table      | Description                                                                                          |
|-------------------|------------------------------------------------------------------------------------------------------|
| `PdM_errors`      | Contains error logs with timestamps and machine IDs.                                                 |
| `PdM_failures`    | Tracks machine failures with timestamps and failure types.                                           |
| `PdM_machines`    | Details about the machines, such as ID, age, and type.                                               |
| `PdM_maint`       | Maintenance records including timestamps and activity types.                                         |
| `PdM_telemetry`   | Time-series data with sensor readings for each machine, such as voltage, rotation, pressure, and vibration. |  



The datasets are sourced from Microsoft Azure Predictive Maintenance Dataset. It can be found on [Kaggle](https://www.kaggle.com/datasets/arnabbiswas1/microsoft-azure-predictive-maintenance/data).  


## 🖥️ **Create Schemas and Tables using DDL**


### **📍1. Schema Creation**

In this project, schemas are being created to logically separate and manage data at different stages of the data processing lifecycle. Each schema serves a specific purpose to ensure data integrity, performance, and maintainability. SQL scripts will be used to define the schema and tables using DDL (Data Definition Language).  

The following schemas were created:

| Schema            | Description                                                                                       |
|--------------------|---------------------------------------------------------------------------------------------------|
| `stg` (staging)  | Acts as a temporary area to store raw data before transformation.                            |
| `dim` (dimension)| Stores descriptive or reference data used for categorization and filtering.                 |
| `f` (fact)       | Centralizes quantitative data like failure counts, error counts, and average telemetry readings. |
 

The following is performed in T-SQL against an instance of MS SQL Server 2022 to create the schemas to be used in this project (stg, dim and f): 

```
-- Creating Dimension Schema
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO
-- Creating Staging Schema
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

GO
-- Creating Fact Schema
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

```  

By using `IF NOT EXISTS`, each schema is created only if it does not already exist in the database.

The datamart will be constructed as the following **Entity Relationship Diagram**:

 ![Entity Relationship Diagram](./images/PdM_ERD.png)    


 Each machine in `dim.Machines` can have multiple records in `dim.Errors`, `dim.Failures`, `dim.Maintenance`, and `f.Telemetry`. So that relationship is one to many.


### **📍2. Dimension tables creation**

The following dimension tables were created to organize and structure the supporting data:  

| Dimension Table     | Description                                                                          |
|---------------------|--------------------------------------------------------------------------------------|
| `dim.Machines`      | Machine details like Machine ID, model, and age.                                    |
| `dim.Errors`        | Logs machine errors.                                                                |
| `dim.Failures`      | Logs machine failures.                                                              |
| `dim.Maintenance`   | Tracks maintenance history for machine components.                                  |
| `dim.Calendar`      | Time dimension for analysis, including date, month, and year.                       |

The SQL scripts for the dimension tables were created with the appropraite data types and keys.

For example the `dim.machines` table was created using the following script:

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Machines')
BEGIN
    CREATE TABLE dim.Machines (
        pkMachineID INT IDENTITY(1,1) NOT NULL,
        MachineID INT NOT NULL,
        Model NVARCHAR(10) NOT NULL,
        Age INT NOT NULL
    );

    ALTER TABLE dim.Machines
    ADD CONSTRAINT PK_Machines PRIMARY KEY (pkMachineID);

    ALTER TABLE dim.Machines
    ADD CONSTRAINT UC_Machines UNIQUE (MachineID);
END;
GO

```   

This script ensures that `dim.Machines` has a primary key (pkMachineID), while also maintaining a unique constraint on MachineID to prevent duplicate machine entries. Additionally, all data types were chosen based on the dataset structure to match the expected data (e.g. INT for IDs and NVARCHAR for text fields).

This was repeated for the other dimension tables with their respective column names.


### **📍3. Fact table creation**


The telemetry becomes a fact table, as it stores measurable events linked to dimensions like machines and time. It can be linked telemetry data to failures, errors, or maintenance events to identify patterns.

| Fact Table    | Description                                                   |
|---------------|---------------------------------------------------------------|
| `f.Telemetry` | Stores telemetry data (performance metrics) for machines.     |
  

For `f.Telemetry`, the table was created with this script:

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'Telemetry')
BEGIN
    CREATE TABLE f.Telemetry (
        TelemetryID INT IDENTITY(1,1) NOT NULL,
        MachineID INT NOT NULL,
        SensorReadTime DATETIME NOT NULL,
        Volt FLOAT NOT NULL,
        Rotate FLOAT NOT NULL,
        Pressure FLOAT NOT NULL,
        Vibration FLOAT NOT NULL
    );

    ALTER TABLE f.Telemetry
    ADD CONSTRAINT PK_Telemetry PRIMARY KEY (TelemetryID);

    ALTER TABLE f.Telemetry
    ADD CONSTRAINT FK_Telemetry_MachineID FOREIGN KEY (MachineID)
    REFERENCES dim.Machines (MachineID);

    ALTER TABLE f.Telemetry
    ADD CONSTRAINT FK_Telemetry_Datetime FOREIGN KEY (datetime)
    REFERENCES dim.Calendar (datetime);
END;
GO

```  

The primary key `TelemetryID` uniquely identifies each record. Foreign Keys, `MachineID` links to `dim.Machines` and `SensorReadTime` to `dim.Calendar` for machine and time-based analysis.



The full SQL DDL scripts for the datamart creation can be found [here](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\SQL_files\PdM_Sql_DDL.sql).

## 🖥️ **Load data into tables using DML**


### **📍1. Load data into staging tables**
Data for staging tables was imported using the [SQL Server Import and Export Wizard](https://github.com/w0435723/PdM_DataMart_Project/blob/main/images/import%2520wizard.png) .The source data files were in CSV format, and each file was mapped to the corresponding staging table under the 'stg' schema. Column mappings and data types were verified during the import process to ensure compatibility with the existing table schema. The import operation used the "Append Rows" option to add data to existing staging tables without overwriting or truncating any previous data.   


### **📍2. Loading Dimension tables**

Dimension tables were populated from staging after verifying that data has been transformed and has no errors.

For `dim.Machines`:

```
INSERT INTO dim.Machines (MachineID, Model, Age)
SELECT 
    s_m.MachineID,
    s_m.Model,
    s_m.Age
FROM stg.Machines AS s_m
WHERE s_m.MachineID NOT IN (SELECT MachineID FROM dim.Machines);
GO

```  

This was repeated for the other dimension tables.

### **📍3. Loading fact table**

Raw data was loaded into the fact table first without aggregation. Aggregated views or summary tables will be created separately so that flexibility is not limited.  

For `f.telemetry`:  

```
INSERT INTO f.Telemetry (MachineID, SensorReadTime, Volt, Rotate, Pressure, Vibration)
SELECT 
    s_t.MachineID,
    s_t.SensorReadTime,
    s_t.Volt,
    s_t.Rotate,
    s_t.Pressure,
    s_t.Vibration
FROM stg.Telemetry AS s_t
WHERE NOT EXISTS (
    SELECT 1 
    FROM f.Telemetry AS f_t
    WHERE s_t.MachineID = f_t.MachineID
      AND s_t.SensorReadTime = f_t.SensorReadTime
);
GO

```  

A test database, called Sandbox, was created to run the SQL scripts and ensure it is runable to load data into the datamart.  


The full SQL DML scripts for loading datamart creation can be found [here](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\SQL_files\PdM_Sql_DML.sql).

**📝 Summary:**

Quick navigation to all other project files can be found at these links:  

- [Readme](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\README.md) 
- [ERD diagram](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\images\PdM_ERD.png)  
- [SQL DDL Scripts](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\SQL_files\PdM_Sql_DDL.sql)   
- [SQL DML Loader](https:\github.com\w0435723\PdM_DataMart_Project\blob\main\SQL_files\PdM_Sql_DML.sql)  

- [Source data files](https:\github.com\w0435723\PdM_DataMart_Project\tree\main\data)  


**📝 References:**

_All emojis in this notebook are sourced from [emojicopy](https:\emojicopy.com\)._

_The datamart was created based on the principles of Star Schema Design as outlined by [Ralph Kimball](https:\www.kimballgroup.com\data-warehouse-business-intelligence-resources\kimball-techniques\dimensional-modeling-techniques\)._  

The source datasets are sourced from Microsoft Azure Predictive Maintenance Dataset on [Kaggle](https://www.kaggle.com/datasets/arnabbiswas1/microsoft-azure-predictive-maintenance/data).